In [ ]:
#from datasets import load_dataset

In [ ]:
#ds = load_dataset("uonlp/CulturaX",
#                  language="en",
#                  token=True, cache_dir='/scratch/hpc-prf-lola/data/culturaX')

In [ ]:
#from huggingface_hub import snapshot_download

In [ ]:
#snapshot_download(repo_id="uonlp/CulturaX", repo_type="dataset", local_dir_use_symlinks=False, local_dir="/scratch/hpc-prf-lola/data/culturaX")

In [ ]:
# Testing the partition of data

In [ ]:
from datasets import load_dataset

In [ ]:
ds1 = load_dataset("jayantdocplix/falcon-small-test-dataset", split='train[0%:15%]')
ds2 = load_dataset("jayantdocplix/falcon-small-test-dataset", split='train[15%:25%]')
ds3 = load_dataset("jayantdocplix/falcon-small-test-dataset", split='train[0%:25%]')

In [ ]:
ds1

In [ ]:
ds2

In [ ]:
ds3

In [ ]:
ds1[-1]

In [ ]:
ds2[0]

In [ ]:
ds3[13994]

In [ ]:
from datasets import load_dataset
ds1 = load_dataset("uonlp/CulturaX", language="en", split='train[0%:1%]')

In [ ]:
ds1

In [ ]:
ds1[32410656]

In [ ]:
import os
import glob
lang_dir = '/scratch/hpc-prf-lola/nikit/.cache/huggingface/datasets--uonlp--CulturaX/snapshots/321a983f3fd2a929cc1f8ef6207834bab0bb9e25'

languages = [f for f in os.listdir(lang_dir) if os.path.isdir(os.path.join(lang_dir, f))]
print(languages)

In [ ]:
from datasets import load_dataset
languages = ['de']
for lang in languages:
    load_dataset("uonlp/CulturaX", language=lang, split='train')

In [ ]:
import os
language = "de"
lang_dir = '/scratch/hpc-prf-lola/nikit/.cache/huggingface/datasets--uonlp--CulturaX/snapshots/321a983f3fd2a929cc1f8ef6207834bab0bb9e25/'+language
lang_parquet_files = [lang_dir + '/' + f for f in os.listdir(lang_dir) if f.endswith(".parquet")]

In [ ]:
lang_parquet_files

In [ ]:
from datasets import load_dataset,Features,Value

context_feat = Features({'text': Value(dtype='string')})

dataset = load_dataset('parquet', features=context_feat, data_files={'train': lang_parquet_files})


In [ ]:
from datasets import load_dataset
import datasets

In [ ]:
dataset = load_dataset('/scratch/hpc-prf-lola/data/raw_datasets/CulturaX', 'pa', split="train")

In [ ]:
dataset

In [ ]:
datasets.config.DEFAULT_MAX_BATCH_SIZE

In [ ]:
dataset.to_json(f"pa.jsonl", batch_size=5000, num_proc=32, orient="records", lines=True, force_ascii=False)

In [ ]:
# Tokenization training

In [ ]:
from datasets import load_dataset
dataset = load_dataset('json', data_files=['/scratch/hpc-prf-lola/data/culturaX/jsonl/af_0-33059_ind.jsonl', '/scratch/hpc-prf-lola/data/culturaX/jsonl/als_0-6935_ind.jsonl'], num_proc=128, keep_in_memory=True)

In [ ]:
iterable_dataset = dataset['train'].to_iterable_dataset(num_shards=128)

In [ ]:
shuffled_iterable_dataset = iterable_dataset.shuffle(seed=42, buffer_size=10000)

In [ ]:
shuffled_iterable_dataset

In [ ]:
for item in shuffled_iterable_dataset:
    print(item)
    break

In [ ]:
len(dataset['train'])

In [ ]:
def dataset_iterator(dataset, text_field):
    for example in dataset:
        yield example[text_field]

In [ ]:
from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()
tokenizer.train_from_iterator(dataset_iterator(shuffled_iterable_dataset, 'text'), length=len(dataset['train']), vocab_size=250000, special_tokens=["<pad>", "<eos>", "<unk>", "<s>", "</s>"], show_progress=True)

In [ ]:
tokenizer

In [ ]:
input_str = "Dié skaapras help haar pa om meer geld uit sy Karakoelpelse te maak en terselfdertyd sy skape se vrugbaarheid te verhoog."
encoding = tokenizer.encode(input_str)
print(encoding)

In [ ]:
encoding.tokens

In [ ]:
decoded_str = tokenizer.decode(encoding.ids)
print(decoded_str)
print(decoded_str == input_str)

In [ ]:
tokenizer.save_model('/scratch/hpc-prf-lola/data/culturaX/tokenizers/temp')

In [ ]:
tokenizer.save('/scratch/hpc-prf-lola/data/culturaX/tokenizers/temp/custom.json')